In [1]:
# Gaussian Elimination Algorithm
# Some parts of the structure and comments were inspired by ChatGPT explanations,
# but the implementation and organization were written and adjusted by me.

import numpy as np

def gaussian_elimination(A, b):
    # Step 1 (from slides): create augmented matrix [A | b]
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    n = len(b)
    aug = np.hstack((A, b.reshape(n, 1)))

    # Step 2 (from slides): forward elimination
    for i in range(n):

        # Pivoting (swap rows if pivot is zero)
        # Idea for checking pivot and swapping was inspired by ChatGPT
        if abs(aug[i, i]) < 1e-12:
            for r in range(i + 1, n):
                if abs(aug[r, i]) > 1e-12:
                    aug[[i, r]] = aug[[r, i]]
                    break

        pivot = aug[i, i]
        if abs(pivot) < 1e-12:
            print("No unique solution (pivot is zero).")
            return None

        # Eliminate entries below pivot
        for j in range(i + 1, n):
            factor = aug[j, i] / pivot
            aug[j, :] = aug[j, :] - factor * aug[i, :]

    # Step 3 (from slides): back substitution
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        if abs(aug[i, i]) < 1e-12:
            print("No unique solution (zero on diagonal).")
            return None

        total = 0
        for j in range(i + 1, n):
            total += aug[i, j] * x[j]

        x[i] = (aug[i, n] - total) / aug[i, i]

    return x


# Example system (same one shown in slides)
# 2x + 3y - z = 5
# 4x - 2y + 2z = 6
# -2x + y + 2z = -1

A = [
    [2, 3, -1],
    [4, -2, 2],
    [-2, 1, 2]
]

b = [5, 6, -1]

solution = gaussian_elimination(A, b)
print("Solution [x, y, z]:", solution)


Solution [x, y, z]: [1.58333333 0.83333333 0.66666667]


In [2]:
# LU Decomposition Algorithm (A = L U)
# Some parts of the structure and comments were inspired by ChatGPT explanations,
# but the implementation and organization were written and adjusted by me.

import numpy as np

def lu_decomposition(A):
    # Slides idea: initialize L as identity, U as a copy of A
    A = np.array(A, dtype=float)
    n = len(A)

    L = np.eye(n)
    U = A.copy()

    # Slides idea: for each column, eliminate below pivot and store multipliers in L
    for i in range(n):
        if abs(U[i, i]) < 1e-12:
            raise ValueError("Zero pivot encountered. LU decomposition cannot proceed.")

        for j in range(i + 1, n):
            factor = U[j, i] / U[i, i]     # multiplier used in elimination
            L[j, i] = factor               # store multiplier in L

            # update row j of U to create zeros below the diagonal
            for k in range(i, n):
                U[j, k] = U[j, k] - factor * U[i, k]

    return L, U

def forward_substitution(L, b):
    L = np.array(L, dtype=float)
    b = np.array(b, dtype=float)
    n = len(b)

    y = np.zeros(n)
    for i in range(n):
        total = 0
        for j in range(i):
            total += L[i, j] * y[j]
        y[i] = (b[i] - total) / L[i, i]
    return y

def back_substitution(U, y):
    U = np.array(U, dtype=float)
    y = np.array(y, dtype=float)
    n = len(y)

    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        total = 0
        for j in range(i + 1, n):
            total += U[i, j] * x[j]
        x[i] = (y[i] - total) / U[i, i]
    return x

def solve_with_lu(A, b):
    # Slides: solve Ly = b then solve Ux = y
    L, U = lu_decomposition(A)
    y = forward_substitution(L, b)
    x = back_substitution(U, y)
    return x, L, U


# Example usage (same system as before)
# 2x + 3y - z = 5
# 4x - 2y + 2z = 6
# -2x + y + 2z = -1

A = [
    [2, 3, -1],
    [4, -2, 2],
    [-2, 1, 2]
]
b = [5, 6, -1]

x, L, U = solve_with_lu(A, b)

print("L matrix:\n", L)
print("U matrix:\n", U)
print("Solution [x, y, z]:", x)

L matrix:
 [[ 1.   0.   0. ]
 [ 2.   1.   0. ]
 [-1.  -0.5  1. ]]
U matrix:
 [[ 2.  3. -1.]
 [ 0. -8.  4.]
 [ 0.  0.  3.]]
Solution [x, y, z]: [1.58333333 0.83333333 0.66666667]
